# Example of Tent Functions for ML

This is a demonstration of how to use tent functions for machine learning on a set of persistence diagrams

In [22]:
import numpy as np
from scipy import stats

from teaspoon.ML.Base import ParameterBucket, build_G, TentParameters, train_test_split, ML_via_featurization
from teaspoon.MakeData import PointCloud as gpc
from scipy import stats
import matplotlib.pyplot as plt

import pandas as pd

## Generate Diagrams from Manifold Test

Generate persistence diagrams drawn from random point clouds of a torus, annulus, cube, three clusters, three clusters of three clusters, and spheres. See details of *testSetManifolds* function [here](http://elizabethmunch.com/code/teaspoon/namespaceteaspoon_1_1_make_data_1_1_point_cloud.html#a5d9c892f9f0a63f64437cbbde9048aeb). Select the dimension of persistence diagram to use, here we use dimension 1.

In [23]:
df = gpc.testSetManifolds(numDgms = 100, numPts = 100)

Generating torus clouds...
Generating annuli clouds...
Generating cube clouds...
Generating three cluster clouds...
Generating three clusters of three clusters clouds...
Generating sphere clouds...
Finished generating clouds and computing persistence.



## Train Test Split & Set up Parameter Bucket
 
### Train Test Split:
 - Choose which column (or columns) you want to use for diagrams
 - Specify which column has the training labels
 
### Parameter Bucket
 - Need a TentParameters parameter bucket. 
 - Set parameter d for mesh size in each direction.
 - Get adaptive partitions. 
 - Set delta and epsilon for each partition.

In [24]:
dgm_col = ['Dgm1']
labels_col = ['trainingLabel']

# Set up parameters and adaptively partition training set
params = TentParameters()
params.useAdaptivePart == False

# Run train/test split using sklearn
D_train, D_test, L_train,L_test = train_test_split(df, df[labels_col], test_size=params.test_size, random_state = params.seed)

# Get the portions of the test data frame with diagrams and concatenate into giant series:
allDgms = pd.concat((D_train[label] for label in dgm_col))

if params.useAdaptivePart == True:
    # Hand the series to the makeAdaptivePartition function
    params.d = [3,3]
    params.makeAdaptivePartition(allDgms, meshingScheme = 'DV', numParts = 3)
else:
    # Just use the bounding box as the partition
    params.d = 3
    params.makeAdaptivePartition(allDgms, meshingScheme = 'None')
    
# Assign delta and epsilon for each partition
# If you didn't use adaptive partitioning this just assigns it to the one partition for the whole bounding box
params.chooseDeltaEpsForPartitions()

Converting the data to ordinal...
Uh oh your support will cross the diagonal, your bottom boundary is  -0.7095354720950127
Shifting the boundary of the partition up by necessary amount...

Partitions d, delta and epsilon have all been assigned to each partition.



## Training

Use function from teaspoon to run ML with featurization on persistence diagrams. Takes data frame of persistence diagrams and specified column labels, computes the G matrix using *build_G*. Does classification using labels from labels_col in the data frame. Returns trained model.

In [25]:
print('Using ' + str(len(L_train)) + '/' + str(len(df)) + ' to train...')
clf = ML_via_featurization(D_train, labels_col = labels_col[0], dgm_col = dgm_col, params = params, verbose = True)

Using 402/600 to train...
Training estimator.
Making G...
Number of features used is 16 ...
Checking score on training set...
Score on training set: 0.8681592039800995.



## Testing

Build G matrix for the testing set, use the model generated on the training data to predict the label. Then score the predicted labels.

In [26]:
#--------Testing-------------#
print('Using ' + str(len(L_test)) + '/' + str(len(df)) + ' to test...')
listOfG = []
for dgmColLabel in dgm_col:
    G = build_G(D_test[dgmColLabel],params)
    listOfG.append(G)

G = np.concatenate(listOfG,axis = 1)

# Compute predictions and add to DgmsDF data frame
L_predict = pd.Series(clf.predict(G),index = L_test.index)
df['Prediction'] = L_predict

# Compute score
score = clf.score(G,list(L_test['trainingLabel']))

print('Score on testing set: ' + str(score) +"...\n")

print('Finished with train/test experiment.')

output = {}
output['score'] = score
output['DgmsDF'] = df
output['clf'] = clf

Using 198/600 to test...
Score on testing set: 0.8787878787878788...

Finished with train/test experiment.
